## CHAPTER 10
# Data Aggregation and Group Operations

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## 10.1 How to Think About Group Operations

In [3]:
df = pd.DataFrame(
    {
        "key1": ["a", "a", None, "b", "b", "a", None],
        "key2": pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
        "data1": np.random.standard_normal(7),
        "data2": np.random.standard_normal(7),
    }
)
df

,key1,key2,data1,data2
0,a,1,1.119043,2.003590
1,a,2,-0.253919,0.713855
2,None,1,0.322419,-0.953233
3,b,2,0.132408,0.105745
4,b,1,1.321117,0.087714
5,a,<NA>,-0.542516,-1.294257
6,None,1,0.643639,-1.243957


In [4]:
grouped = df["data1"].groupby(df["key1"])
grouped

In [5]:
grouped.mean()

key1
a    0.107536
b    0.726763
Name: data1, dtype: float64

In [6]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1       1.119043
      2      -0.253919
b     1       1.321117
      2       0.132408
Name: data1, dtype: float64

In [7]:
means.unstack()

key2,1,2
key1,,
a,1.119043,-0.253919
b,1.321117,0.132408


In [8]:
state = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = np.array([2005, 2005, 2006, 2005, 2006, 2005, 2006])
df["data1"].groupby([state, years]).mean()

CA  2005   -0.398217
    2006    0.322419
OH  2005    0.625726
    2006    0.982378
Name: data1, dtype: float64

In [9]:
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,0.107536,0.474396
b,1.5,0.726763,0.096730


In [10]:
df.groupby("key2").mean(numeric_only=True)

,data1,data2
key2,,
1,0.851554,-0.026471
2,-0.060755,0.409800


In [11]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1     1.119043  2.003590
     2    -0.253919  0.713855
b    1     1.321117  0.087714
     2     0.132408  0.105745

In [12]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [13]:
df.groupby("key1", dropna=False).size()

key1
a      3
b      2
NaN    2
dtype: int64

In [14]:
df.groupby(["key1", "key2"], dropna=False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [15]:
df.groupby("key1").count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


### Iterating over Groups

In [16]:
for name , group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1  1.119043  2.003590
1    a     2 -0.253919  0.713855
5    a  <NA> -0.542516 -1.294257
b
  key1  key2     data1     data2
3    b     2  0.132408  0.105745
4    b     1  1.321117  0.087714


In [17]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', np.int64(1))
  key1  key2     data1    data2
0    a     1  1.119043  2.00359
('a', np.int64(2))
  key1  key2     data1     data2
1    a     2 -0.253919  0.713855
('b', np.int64(1))
  key1  key2     data1     data2
4    b     1  1.321117  0.087714
('b', np.int64(2))
  key1  key2     data1     data2
3    b     2  0.132408  0.105745


In [18]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces["b"]

,key1,key2,data1,data2
3,b,2,0.132408,0.105745
4,b,1,1.321117,0.087714


In [19]:
pieces

{'a':   key1  key2     data1     data2
 0    a     1  1.119043  2.003590
 1    a     2 -0.253919  0.713855
 5    a  <NA> -0.542516 -1.294257,
 'b':   key1  key2     data1     data2
 3    b     2  0.132408  0.105745
 4    b     1  1.321117  0.087714}

In [20]:
grouped = df.groupby({"key1": "key", "key2": "key",
                      "data1": "data", "data2": "data"}, axis="columns")
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

data
      data1     data2
0  1.119043  2.003590
1 -0.253919  0.713855
2  0.322419 -0.953233
3  0.132408  0.105745
4  1.321117  0.087714
5 -0.542516 -1.294257
6  0.643639 -1.243957
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


C:\Users\sahor\AppData\Local\Temp\ipykernel_7736\695880334.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped = df.groupby({"key1": "key", "key2": "key",


### Selecting a Column or Subset of Columns

In [21]:
df

,key1,key2,data1,data2
0,a,1,1.119043,2.003590
1,a,2,-0.253919,0.713855
2,None,1,0.322419,-0.953233
3,b,2,0.132408,0.105745
4,b,1,1.321117,0.087714
5,a,<NA>,-0.542516,-1.294257
6,None,1,0.643639,-1.243957


In [22]:
df.groupby("key1")["data1"]
df.groupby("key1")[["data2"]]

In [23]:
df["data1"].groupby(df["key1"])
df[["data2"]].groupby(df["key1"])

In [24]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1     2.003590
     2     0.713855
b    1     0.087714
     2     0.105745

In [25]:
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped

In [26]:
s_grouped.mean()

key1  key2
a     1       2.003590
      2       0.713855
b     1       0.087714
      2       0.105745
Name: data2, dtype: float64

### Grouping with Dictionaries and Series

In [27]:
people = pd.DataFrame(
    np.random.randn(5, 5),
    columns=["a", "b", "c", "d", "e"],
    index=["Joe", "Steve", "Wanda", "Jill", "Trey"],
)
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,-1.511647,1.328566,-0.464251,0.322950,0.584419
Steve,0.640762,-1.183368,-0.617839,-1.148129,1.459241
Wanda,0.528977,NaN,NaN,-0.447840,1.386639
Jill,-0.197512,-0.701489,-0.051202,-1.091185,0.069543
Trey,-2.492794,-1.217849,1.418743,0.865782,0.380525


In [28]:
mapping = {"a": "red", "b": "red", "c": "blue",
            "d": "blue", "e": "red", "f" : "orange"}


In [29]:
by_column = people.groupby(mapping, axis="columns")
by_column.sum()

C:\Users\sahor\AppData\Local\Temp\ipykernel_7736\2628240624.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  by_column = people.groupby(mapping, axis="columns")


,blue,red
Joe,-0.141301,0.401338
Steve,-1.765969,0.916635
Wanda,-0.447840,1.915615
Jill,-1.142387,-0.829458
Trey,2.284525,-3.330117


In [30]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [31]:
people.groupby(map_series, axis="columns").count()

C:\Users\sahor\AppData\Local\Temp\ipykernel_7736\2727320543.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  people.groupby(map_series, axis="columns").count()


,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


### Grouping with Functions

In [32]:
people

,a,b,c,d,e
Joe,-1.511647,1.328566,-0.464251,0.322950,0.584419
Steve,0.640762,-1.183368,-0.617839,-1.148129,1.459241
Wanda,0.528977,NaN,NaN,-0.447840,1.386639
Jill,-0.197512,-0.701489,-0.051202,-1.091185,0.069543
Trey,-2.492794,-1.217849,1.418743,0.865782,0.380525


In [33]:
people.groupby(len).sum()  # group by length of the index value

,a,b,c,d,e
3,-1.511647,1.328566,-0.464251,0.322950,0.584419
4,-2.690305,-1.919338,1.367541,-0.225404,0.450068
5,1.169739,-1.183368,-0.617839,-1.595970,2.845880


In [34]:
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,-1.511647,1.328566,-0.464251,0.322950,0.584419
4,two,-2.492794,-1.217849,-0.051202,-1.091185,0.069543
5,one,0.528977,-1.183368,-0.617839,-1.148129,1.386639


### Grouping by Index Levels

In [35]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
                                     [1, 3, 5, 1, 3]],
                                    names=["cty", "tenor"])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.745440  0.471702  0.676751 -2.068494 -0.702796
1      2.369008 -1.261372 -0.394469  1.396463 -0.041104
2      0.541585  1.376963  0.113311  0.051306  0.136304
3      0.206527 -0.448298  0.348606 -0.349054 -1.056049

In [36]:
hier_df.groupby(level="cty", axis="columns").count()

C:\Users\sahor\AppData\Local\Temp\ipykernel_7736\2795167867.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  hier_df.groupby(level="cty", axis="columns").count()


cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 Data Aggregation

In [37]:
df

,key1,key2,data1,data2
0,a,1,1.119043,2.003590
1,a,2,-0.253919,0.713855
2,None,1,0.322419,-0.953233
3,b,2,0.132408,0.105745
4,b,1,1.321117,0.087714
5,a,<NA>,-0.542516,-1.294257
6,None,1,0.643639,-1.243957


In [38]:
grouped = df.groupby("key1")
grouped["data1"].nsmallest(2)

key1   
a     5   -0.542516
      1   -0.253919
b     3    0.132408
      4    1.321117
Name: data1, dtype: float64

In [39]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,1.661558,3.297847
b,1,1.188709,0.018031


In [40]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.107536  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0  0.726763  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     0.432562  1.119043   3.0  0.474396  1.661913 -1.294257 -0.290201   
b     1.023940  1.321117   2.0  0.096730  0.012750  0.087714  0.092222   

                                    
           50%       75%       max  
key1                                
a     0.713855  1.358723  2.003590  
b     0.096730  0.101237  0.105745  

[2 rows x 24 columns]

### Column-Wise and Multiple Function Application

In [41]:
# tips = sns.load_dataset("tips")
tips = pd.read_csv("examples/tips.csv")
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [42]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [43]:
grouped = tips.groupby(["day", "smoker"])
grouped_pct = grouped["tip_pct"]
grouped_pct.mean()

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [44]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [45]:
grouped_pct.agg([("average", "mean"), ("stdev", "std")])

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [46]:
functions = ["count", "mean", "max"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [47]:
result["tip_pct"]

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [48]:
ftuples = [("Average", "mean"), ("Variance", np.var)]
grouped[["tip_pct", "total_bill"]].agg(ftuples)

C:\Users\sahor\AppData\Local\Temp\ipykernel_7736\4198102554.py:2: FutureWarning: The provided callable <function var at 0x0000021896E07B00> is currently using SeriesGroupBy.var. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "var" instead.
  grouped[["tip_pct", "total_bill"]].agg(ftuples)


tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [49]:
grouped.agg({"tip": np.max, "size": "sum"})

C:\Users\sahor\AppData\Local\Temp\ipykernel_7736\3453032214.py:1: FutureWarning: The provided callable <function max at 0x0000021896E06E80> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  grouped.agg({"tip": np.max, "size": "sum"})


tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [50]:
grouped.agg({"tip_pct": ["min", "max", "mean", "std"],
            "size": "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes

In [52]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [ ]:
tips.groupby(["day", "smoker"], as_index=False).mean(numeric_only=True) # Added numeric_only=True to avoid FutureWarning # শুধু numeric কলাম নাও

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863
